<a href="https://colab.research.google.com/github/SourCherries/VAE-GAN-CelebA/blob/master/visualize_celeba_15_AFC_sheldon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/SourCherries/VAE-GAN-CelebA.git

%tensorflow_version 1.x

import sys, getopt
if sys.version_info < (3,4):
    print('You are running an older version of Python!\n\n',
          'You should consider updating to Python 3.4.0 or',
          'higher.\n')
print(sys.version)

sys.path.insert(0,'/content/VAE-GAN-CelebA/libs')
sys.path.insert(0,'/content/VAE-GAN-CelebA/')

import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import libs.make_network as make_network
import libs.utils as utils
from scipy.io import loadmat, savemat
import imageio
from skimage.util import montage
import pandas as pd
import seaborn as sns

import zipfile
import io
from PIL import Image
from torchvision import transforms

print('TF version = ',tf.__version__)

# Download Tensorflow checkpoint
! gdown --id 1o9nUhtZEX9GZXsTmvCXSNjEjDQ7Q1wHV

# CelebA
# https://drive.google.com/file/d/172v_Sfx-MyE2nS59jxl6W9-so3Xvyhkj/view?usp=sharing
! gdown --id 172v_Sfx-MyE2nS59jxl6W9-so3Xvyhkj



## Model

In [ ]:
sess, X, G, Z, Z_mu, is_training, saver = make_network.make_network()

if os.path.exists("vaegan_celeba.ckpt"):
    saver.restore(sess, "vaegan_celeba.ckpt")
    print("VAE-GAN model restored.")
else:
    print("Pre-trained network appears to be missing.")
    sys.exit()



## Mean Z for 15 identities
Final is Sheldon, first 7 are most similar to Sheldon, and next 7 have mean Z that are orthogonal to Sheldon.

In [ ]:
z_file = "celeba-15-AFC-sheldon-last"
Z = np.fromfile(z_file)

ID = Z[:, 0]
Z = Z[:, 1:]

## Batchwise convert to images

In [ ]:
def imdeprocess(g):
    stretch = 1.0
    for i in range(g.shape[0]):
        g[i]=np.clip(stretch*g[i] / (g.max()),0,1)
    return g

g = sess.run(G, feed_dict={Z: (1*Z), is_training: False})

for gg in g:
  img = imdeprocess(gg)
  plt.imshow(img); plt.show()